In [1]:
from significance_analysis import *
from pymer4.models import Lmer
import bambi
import arviz as az
import pandas as pd

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [6]:
data = pd.read_parquet("significance_analysis_example/datasets/priorband_data.parquet")
data=data.loc[(data["seed"]<10) & (data["used_fidelity"]==10)]#& (data["benchmark"].isin(["JAHS-FM","JAHS-CH"]))]

In [7]:
lmem=Lmer("value ~ algorithm + (1|benchmark)",data)
lmem.fit(factors={"algorithm":list(data["algorithm"].unique())})
print(lmem.summary())
print(lmem.post_hoc(marginal_vars=["algorithm"]))

c:\Users\Amega\Git\lmem-significance\.venv_3_10_0\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: value~algorithm+(1|benchmark)

Family: gaussian	 Inference: parametric

Number of observations: 720	 Groups: {'benchmark': 12.0}

Log-likelihood: -192.231 	 AIC: 394.463

Random effects:

                  Name     Var    Std
benchmark  (Intercept)  10.514  3.243
Residual                 0.085  0.292

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: value~algorithm+(1|benchmark)

Family: gaussian	 Inference: parametric

Number of observations: 720	 Groups: {'benchmark': 12.0}

Log-likelihood: -192.231 	 AIC: 394.463

Random effects:

                  Name     Var    Std
benchmark  (Intercept)  10.514  3.243
Residual                 0.085  0.292

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  P-val Sig
(Intercept)     1.944   0.109    3.779  0.936   11.006   2.076  0.062   .
algorithm1      0.042  -0.01

In [8]:
bamb=bambi.Model("value ~ algorithm + (1|benchmark)",data)
print(bamb)
results=bamb.fit()
print(results)


       Formula: value ~ algorithm + (1|benchmark)
        Family: gaussian
          Link: mu = identity
  Observations: 720
        Priors: 
    target = mu
        Common-level effects
            Intercept ~ Normal(mu: 1.976, sigma: 11.0245)
            algorithm ~ Normal(mu: [0. 0.], sigma: [16.5368 16.5368])
        
        Group-level effects
            1|benchmark ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 11.0245))
        
        Auxiliary parameters
            sigma ~ HalfStudentT(nu: 4.0, sigma: 3.1182)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, algorithm, 1|benchmark_sigma, 1|benchmark_offset]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 65 seconds.
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data


In [9]:
az.summary(results)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
1|benchmark[JAHS-C10],8.063,1.083,6.042,10.249,0.048,0.035,510.0,655.0,1.01
1|benchmark[JAHS-CH],3.625,1.082,1.588,5.786,0.048,0.035,511.0,642.0,1.01
1|benchmark[JAHS-FM],3.232,1.082,1.216,5.406,0.048,0.035,510.0,620.0,1.01
1|benchmark[LC-126026],-1.985,1.083,-4.054,0.154,0.048,0.034,511.0,639.0,1.01
1|benchmark[LC-167190],-1.838,1.083,-3.872,0.323,0.048,0.034,510.0,638.0,1.01
1|benchmark[LC-168330],-1.623,1.083,-3.646,0.549,0.048,0.034,512.0,633.0,1.01
1|benchmark[LC-168910],-1.723,1.083,-3.675,0.530,0.048,0.034,510.0,645.0,1.01
1|benchmark[LC-189906],-1.876,1.083,-3.931,0.279,0.048,0.034,511.0,635.0,1.01
1|benchmark[PD1-Cifar100],-1.739,1.083,-3.794,0.398,0.048,0.034,512.0,630.0,1.01
1|benchmark[PD1-ImageNet],-1.756,1.082,-3.815,0.382,0.048,0.034,511.0,636.0,1.01
